___
# Importing necessary libraries
___

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')


import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px


import cufflinks as cf
cf.go_offline()

In [2]:
# Loading the dataset
df = pd.read_csv('../dataset/COVID-19_Sentiments.csv')

In [3]:
df.head()

,Text_Id,Text,Date,Location,Sentiments
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,"Uttar Pradesh, India",0.000
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,Mumbai,0.000
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",Fri Mar 20 16:04:29 +0000 2020,"Jodhpur, India",0.350
3,1241032877099237379,RT @narendramodi: The young actors have someth...,Fri Mar 20 16:04:29 +0000 2020,"Gurugram, Bharat",0.125
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:28 +0000 2020,"New Delhi, India",0.000


In [4]:
df.drop('Text_Id' , axis = 1 , inplace = True)

In [9]:
 # Retaining only date in Date feature
df['Date'] = pd.to_datetime( df['Date'] ).dt.date

In [10]:
df.head()

,Text,Date,Location,Sentiments
0,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000
1,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000
2,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350
3,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125
4,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000


In [11]:
lis = []
for value in df.Sentiments:
    if value < 0 :
        lis.append(-1)
    elif value == 0 :
        lis.append(0)
    else:
        lis.append(1)
df['Sentiment Score'] = lis 
df.head()